In [58]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import torch
import numpy as np
from torch_geometric.data import Data
import anndata
# import os
import hdf5plugin
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import numpy as np
np.random.seed(42)  # 예시로 42를 시드로 설정


In [60]:
biogrid = pd.read_csv("/home/hb/lof/network/biogrid/integrated_network_dropdp_sorted_gene_set.csv")
gene_label_dict = pd.read_pickle("/home/hb/lof/code/deg_augumentation/gene_label_dict.pickle")
gene_dict = pd.read_pickle("/home/hb/lof/code/deg_augumentation/gene_dict.pickle")

In [75]:
biogrid = biogrid[biogrid["gene_symbol_A"].isin(gene_label_dict.values())]
biogrid = biogrid[biogrid["gene_symbol_B"].isin(gene_label_dict.values())]
biogrid['label_A'] = biogrid['gene_symbol_A'].map(gene_dict)
biogrid['label_B'] = biogrid['gene_symbol_B'].map(gene_dict)
biogrid['edge'] = biogrid['label_A'].astype(str) + '-' + biogrid['label_B'].astype(str)
edge_index = torch.tensor(np.vstack([biogrid['label_A'].values, biogrid['label_B'].values]))

In [77]:
gene_set = list(gene_dict.keys())

In [79]:
import os

len(os.listdir("/data_hdd4/hb/lof/data/deg_augmentation/after_labeling"))


93091

In [87]:
os.listdir("/data_hdd4/hb/lof/data/deg_augmentation/after_labeling")[0].split('_')[:-1]

['PTPN13']

In [180]:
import os
from tqdm import tqdm
import pickle

DIM = len(gene_set)
data_save_dir = "/data_hdd4/hb/lof/data/deg_augmentation/after_labeling"
for _data in tqdm(os.listdir(data_save_dir)):
    if len(_data.split('_')) > 2 :
        pb_set = _data.split('_')[:-1]
        pb_set = [x if x in gene_dict.keys() else 0 for x in pb_set]
    else:
        pb_set = _data.split('_')[0]
        pb_set = [pb_set]
        pb_set = [x if x in gene_dict.keys() else 0 for x in pb_set]

    if 0 not in pb_set:
        print('ALL PERTURB GENES IN DICT!', pb_set)
        # DEG 데이터 불러오기
        _df = pd.read_pickle(os.path.join(data_save_dir, _data))
        # gene_symbol을 숫자로 labeling / gene_dict에 없을 경우 nan처리하여 지우기
        _df['gene_label'] = _df['gene_name'].apply(lambda x: int(gene_dict[x]) if x in gene_dict else float('nan'))
        _df = _df.dropna(subset=['gene_label'], ignore_index=True).sort_values(by='gene_label')
        # X, Y 데이터 생성
            # X 데이터 생성 시 없는 deg value는 0으로 채워넣기
        x_dict = dict(zip([int(x) for x in _df['gene_label']], _df.iloc[:, -2].values))
        not_in_dict = set(range(DIM)) - set(x_dict.keys())
        x_dict.update(dict(zip(not_in_dict, [0]*len(not_in_dict))))
        _x = np.array(list(x_dict.values())).reshape(-1, 1)
        perturb_gene_label = [gene_dict[x] for x in pb_set]
        _y = np.zeros(DIM)
        for _gene in perturb_gene_label:
            _y[_gene] = 1

        data = Data(x=_x, edge_index=edge_index, y=_y)
        with open(f'/data_hdd4/hb/lof/data/norman_input_data_pickle/{_data}', 'wb') as f:
            pickle.dump(data, f)


  0%|          | 6/93091 [00:00<26:51, 57.75it/s]

ALL PERTURB GENES IN DICT! ['SAMD1', 'PTPN12']
ALL PERTURB GENES IN DICT! ['UBASH3B', 'OSR2']
ALL PERTURB GENES IN DICT! ['KLF1', 'CEBPA']
ALL PERTURB GENES IN DICT! ['S1PR2']
ALL PERTURB GENES IN DICT! ['MAML2']
ALL PERTURB GENES IN DICT! ['ETS2', 'MAPK1']
ALL PERTURB GENES IN DICT! ['PRDM1', 'CBFA2T3']
ALL PERTURB GENES IN DICT! ['KLF1']
ALL PERTURB GENES IN DICT! ['FOXA3', 'HOXB9']
ALL PERTURB GENES IN DICT! ['AHR', 'KLF1']
ALL PERTURB GENES IN DICT! ['BPGM']


  0%|          | 19/93091 [00:00<26:07, 59.37it/s]

ALL PERTURB GENES IN DICT! ['MAP7D1']
ALL PERTURB GENES IN DICT! ['PTPN1']
ALL PERTURB GENES IN DICT! ['IGDCC3', 'MAPK1']
ALL PERTURB GENES IN DICT! ['KIF18B']
ALL PERTURB GENES IN DICT! ['ETS2', 'MAPK1']
ALL PERTURB GENES IN DICT! ['MAP2K3']
ALL PERTURB GENES IN DICT! ['MAPK1', 'IKZF3']
ALL PERTURB GENES IN DICT! ['IRF1']
ALL PERTURB GENES IN DICT! ['ZBTB10', 'PTPN12']
ALL PERTURB GENES IN DICT! ['CBFA2T3']
ALL PERTURB GENES IN DICT! ['KLF1', 'CLDN6']
ALL PERTURB GENES IN DICT! ['MAPK1', 'TGFBR2']


  0%|          | 31/93091 [00:00<27:37, 56.13it/s]

...

 18%|█▊        | 17025/93091 [04:28<25:08, 50.43it/s]

ALL PERTURB GENES IN DICT! ['PTPN1']
ALL PERTURB GENES IN DICT! ['PTPN1']
ALL PERTURB GENES IN DICT! ['KIF2C']
ALL PERTURB GENES IN DICT! ['SET']
ALL PERTURB GENES IN DICT! ['CBL', 'UBASH3B']
ALL PERTURB GENES IN DICT! ['KMT2A']
ALL PERTURB GENES IN DICT! ['MAPK1']
ALL PERTURB GENES IN DICT! ['UBASH3B']
ALL PERTURB GENES IN DICT! ['ETS2']


 18%|█▊        | 17039/93091 [04:28<23:29, 53.94it/s]

ALL PERTURB GENES IN DICT! ['MAPK1']
ALL PERTURB GENES IN DICT! ['LHX1']
ALL PERTURB GENES IN DICT! ['CBL']
ALL PERTURB GENES IN DICT! ['ZBTB10']
ALL PERTURB GENES IN DICT! ['MAP4K5']
ALL PERTURB GENES IN DICT! ['TMSB4X']
ALL PERTURB GENES IN DICT! ['MAPK1']
ALL PERTURB GENES IN DICT! ['TGFBR2']
ALL PERTURB GENES IN DICT! ['SAMD1', 'UBASH3B']
ALL PERTURB GENES IN DICT! ['BPGM']
ALL PERTURB GENES IN DICT! ['S1PR2']


 18%|█▊        | 17045/93091 [04:29<24:11, 52.37it/s]

In [ ]:
import os
from tqdm import tqdm
DIM = len(gene_set)
for _perturb in tqdm(os.listdir("/data_hdd4/hb/lof/data/deg_augmentation")):
    _path = os.path.join("/data_hdd4/hb/lof/data/deg_augmentation", _perturb)
    if '_' in _perturb:
        perturb_set = _perturb.split('_')
    else:
        perturb_set = [_perturb]

    perturb_set = [x  if x in gene_set else 0 for x in perturb_set]

    if 0 not in perturb_set:
        print('ALL PERTURB GENES IN DICT!', perturb_set)

        for _data_point in tqdm(os.listdir(_path)):
            _df = pd.read_csv(os.path.join(_path, _data_point))
            _df['labels'] = _df['gene_name'].apply(lambda x: int(gene_label_dict[x]) if x in gene_label_dict else float('nan'))
            _df = _df.dropna(subset=['labels'], ignore_index=True).sort_values(by='labels')
            x_dict = dict(zip([int(x) for x in _df['labels']], _df.iloc[:, -2].values))
            not_in_dict = set(range(DIM)) - set(x_dict.keys())
            x_dict.update(dict(zip(not_in_dict, [0]*len(not_in_dict))))
            _x = np.array(list(x_dict.values()))
            X.append(_x)
            
            perturb_gene_label = [gene_label_dict[x] for x in perturb_set]
            _y = np.zeros(DIM)
            for _gene in perturb_gene_label:
                _y[_gene] = 1
            y.append(_y)
    else:
        print('pass! ', perturb_set)
    print('==='*30)

In [32]:
DIM = len(gene_list)



ahr_x1 = pd.read_csv("/data_hdd4/hb/lof/data/deg_augmentation/AHR/AHR_1337.csv")
_df = ahr_x1
_df['gene_label'] = _df['gene_name'].map(gene_dict)
_df = _df.dropna(subset=['gene_label'], ignore_index=True).sort_values(by='gene_label')
_df['gene_label'] = _df['gene_label'].astype(int)

x_dict = dict(zip(_df['gene_label'].values, _df.iloc[:, -2].values))
not_in_dict = set(range(DIM)) - set(x_dict.keys())
not_in_dict = dict(zip(not_in_dict, [0]*len(not_in_dict)))
x_dict.update(not_in_dict)
len(x_dict.keys())
x = torch.tensor(list(x_dict.values()), dtype=torch.float32).view(-1, 1)
x.shape

torch.Size([14592, 1])

In [33]:
perturb_set = ['AHR']
perturb_gene_label = [gene_dict[x] for x in perturb_set]
_y = np.zeros(DIM)
for _gene in perturb_gene_label:
    _y[_gene] = 1

In [34]:
biogrid_data = Data(x=x, edge_index=edge_index, y=_y)
biogrid_data

Data(x=[14592, 1], edge_index=[2, 743637], y=[14592])

In [ ]:
"""
1. 완전 랜덤하게 20%
2. unseen perturb case도 해볼 수 있게 데이터셋 생성
"""

"""DEG info가 없는 경우
MIT- gene들이 전체 read의 몇 %이상이면 quality.. 

"""

In [35]:
torch.save(biogrid_data, '/data_hdd4/hb/lof/data/norman_input_data/ahr_x3.pt')

In [36]:
aab = torch.load('/data_hdd4/hb/lof/data/norman_input_data/ahr_x3.pt')
aab

Data(x=[14592, 1], edge_index=[2, 743637], y=[14592])

In [8]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    batch

    batch.num_graphs


/home/hb/anaconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
class PhosphoDataset(Dataset):
    """Face Landmarks dataset."""
    

    def __init__(self, pkl_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        if isinstance(pkl_file, str):
            self.dataset_frame = pd.read_pickle(pkl_file)
        elif isinstance(pkl_file, pd.DataFrame):
            self.dataset_frame = pkl_file
        else:
            raise ValueError('pkl_file must be a str or a pandas dataframe')
        self.dataset_frame = self.dataset_frame.sample(frac=1, random_state=42).reset_index(drop=True)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.dataset_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        pair_id = self.dataset_frame.pair_id[idx]
        matrix_name = os.path.join(self.root_dir, pair_id)
        matrix = np.load(f'{matrix_name}.npy')
        matrix = torch.from_numpy(matrix)
        label = self.dataset_frame.answer[idx]
        if 'startify' in self.dataset_frame.columns:
            stratify = self.dataset_frame.stratify[idx]
            sample = {'id': pair_id, 'matrix': matrix, 'label': label, 'stratify': stratify}
        else:
            sample = {'id': pair_id, 'matrix': matrix, 'label': label}
        return sample

In [27]:
import os.path as osp
import os
import torch
from torch_geometric.data import Dataset, download_url


class DegDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.raw_paths = root

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data_1.pt', 'data_2.pt', ...]


    def process(self):
        idx = 0     
        for raw_path in os.listdir(self.raw_paths):
            # Read data from `raw_path`.
            data = torch.load(osp.join(self.raw_paths, raw_path))
            print(data)
            if self.pre_filter is not None and not self.pre_filter(data):
                continue

            if self.pre_transform is not None:
                data = self.pre_transform(data)

            torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
            idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [28]:
tt = DegDataset(root='/data_hdd4/hb/lof/data/norman_input_data/')
tt

DegDataset(3)

In [31]:
len(tt)

3